<a href="https://colab.research.google.com/github/joonhahaileypark/joonhahaileypark/blob/main/labs/de_lab_1_Joonha_Park.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Engineering - Lab 1


## Intro to table of contents

In this lab we'll be introduced to the Google Colab environment. To start this notebook simply go to "Runtime" and choose "Run All". This will connect Colab to your virtual machine and run the code needed to get your database setup.

You'll notice that notebooks are setup with a table of contents navigation bar on the left nav bar of your screen. Mouse over and hover to see which options are available and click "Table of Contents" to see the major sections to this Notebook.  

## Database setups (do not modify)

In [2]:
# setups including getting the database

!pip -q install --upgrade ipython
!pip -q install --upgrade ipython-sql

!wget -O northwind.db https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db

import sqlite3
con = sqlite3.connect("northwind.db")

%load_ext sql
%sql sqlite:///northwind.db

%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

--2025-02-20 04:16:22--  https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db [following]
--2025-02-20 04:16:22--  https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 602112 (588K) [application/octet-stream]
Saving to: ‘northwind.db’

northwind.db        100%[===================>] 588.00K  3.12MB/s    in 0.2s    

2025-02-20 04:16:23 (3.12 MB/s) - ‘northwind.db’ saved [602112/602112]

Th

## ERD of the Northwind Database


![ERD](https://github.com/matthewpecsok/data_engineering/blob/main/data/Northwind_ERD.png?raw=true)


Using %sql allows for one line sql statements
Using %%sql allows for multi-line sql statements and better readability for long sql queries.

In [3]:
%sql select * from products limit 4

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22,53,0,0,0


In [4]:
%%sql

select
count(1) as product_count
from
products

,product_count
0,77


## Lab question 1

In [6]:
%%sql
Select
count(1) as unitprice_gt_18
from Products where UnitPrice > 18

,unitprice_gt_18
0,43


## Lab question 2


In [8]:
%%sql
SELECT c.CompanyName, COUNT(o.OrderID) AS order_count
FROM Orders o
JOIN Customers c ON o.CustomerID = c.CustomerID
GROUP BY c.CompanyName
ORDER BY order_count DESC
LIMIT 5;


,CompanyName,order_count
0,Save-a-lot Markets,31
1,Ernst Handel,30
2,QUICK-Stop,28
3,Hungry Owl All-Night Grocers,19
4,Folk och fä HB,19


## Lab question 3



Create a dataset showing the OrderId and CategoryName for all orders in the database. Limit the result set to 5 rows. Order by CategoryId.
To form this query you'll need to join multiple tables together. Order Details has a space in the name. You can wrap this table in quotes to handle the space in the name. "Order Details"

In [10]:
%%sql
SELECT od.OrderID, c.CategoryName
FROM "Order Details" od
JOIN Products p ON od.ProductID = p.ProductID
JOIN Categories c ON p.CategoryID = c.CategoryID
ORDER BY c.CategoryID
LIMIT 5;


,OrderID,CategoryName
0,10253,Beverages
1,10254,Beverages
2,10255,Beverages
3,10257,Beverages
4,10258,Beverages


## Lab question 4



Show the counts of all orders in the database grouped by ShipRegion.
The columns should be named RegionCount and ShipRegion respectively.
Filter this dataset to include only counts greater than 100.
Sort by the count in descending order.

In [31]:
%%sql
SELECT COUNT(OrderID) AS RegionCount, ShipRegion
FROM Orders
GROUP BY ShipRegion
HAVING COUNT(OrderID) > 100
ORDER BY RegionCount DESC;





,RegionCount,ShipRegion
0,276,Western Europe
1,152,North America
2,145,South America


add additional sections as needed to complete the lab

# Lab Question 5
Create a single query showing the max, min and average unit price from the products table.
Round the average price to 2 decimal places.
In the query, rename the average column name to average_unit_price

In [35]:
%%sql
SELECT
    MAX(UnitPrice) AS max_unit_price,
    MIN(UnitPrice) AS min_unit_price,
    ROUND(AVG(UnitPrice), 2) AS average_unit_price
FROM products;


,max_unit_price,min_unit_price,average_unit_price
0,263.5,2.5,28.87


#Lab Question 6
Create a query returning 4 columns
1) the original_total column which is the unitprice*quantity from the order_detail table
2) the discounted_price column which is unitprice*quantity - unitprice*quantity* discount
3) the discount amount
4) The discount percentage
4) the company name (via customer table) Finally: filter the dataset for CompanyName Ernst Handel and original_total > 4000
Your output should look as follows:

In [43]:
%%sql
SELECT
    od.UnitPrice * od.Quantity AS original_total,
    (od.UnitPrice * od.Quantity) - (od.UnitPrice * od.Quantity * od.Discount) AS discounted_price,
    (od.UnitPrice * od.Quantity * od.Discount) AS discount_amount,
    od.Discount AS discount_percentage,
    c.CompanyName
FROM "Order Details" od
JOIN Orders o ON od.OrderID = o.OrderID
JOIN Customers c ON o.CustomerID = c.CustomerID
WHERE c.CompanyName = 'Ernst Handel'
AND (od.UnitPrice * od.Quantity) > 4000;


,original_total,discounted_price,discount_amount,discount_percentage,CompanyName
0,4216.0,4005.2,210.8,0.05,Ernst Handel
1,6360.0,6042.0,318.0,0.05,Ernst Handel
2,6050.0,6050.0,0.0,0.00,Ernst Handel
3,4322.5,4322.5,0.0,0.00,Ernst Handel


#Lab Question 7
Create a query returning the orderdate formatted as a year named order_year, the count of orders named order_count grouped by year and ordered by order_year.
This query should return 3 rows 1 for each year with the count for each year respectively.

In [57]:
%%sql
SELECT
    SUBSTR(OrderDate, 1, 4) AS order_year,
    COUNT(*) AS order_count
FROM Orders
GROUP BY order_year
ORDER BY order_year;


,order_year,order_count
0,2016,152
1,2017,408
2,2018,270


#Question 8


In [59]:
%%sql
SELECT
    e.EmployeeID,
    e.FirstName || ' ' || e.LastName AS EmployeeName,
    COUNT(o.OrderID) AS TotalOrders
FROM Employees e
JOIN Orders o ON e.EmployeeID = o.EmployeeID
GROUP BY e.EmployeeID, EmployeeName
ORDER BY TotalOrders DESC
LIMIT 5;


,EmployeeID,EmployeeName,TotalOrders
0,4,Margaret Peacock,156
1,3,Janet Leverling,127
2,1,Nancy Davolio,123
3,8,Laura Callahan,104
4,2,Andrew Fuller,96


#question 9


In [63]:
%%sql SELECT COUNT(*) FROM sqlite_master
WHERE type='table';

,COUNT(*)
0,14


#Question 10

In [66]:
%sql select COUNT(*) AS view_count from sqlite_master where type = 'view'

,view_count
0,17


#Question 11
Left Join the customer table to the orders table with a left join, get the count of the rows.Links to an external site.

In [73]:
%%sql
SELECT COUNT(*)
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID;


,COUNT(*)
0,834


#Question 12
Join the customer table to the orders table with a inner join, get the count of the rows. in a text block explain why the count is different.
You may need to create additional queries to answer this question

In [74]:
%%sql
SELECT COUNT(*)
FROM Customers c
INNER JOIN Orders o ON c.CustomerID = o.CustomerID;

,COUNT(*)
0,830


#Question 13
Create a new table in the database. Give it a name and create 3 columns of your choosing. Write a query to return the count of rows in the table.

In [80]:
%%sql
CREATE TABLE IF NOT EXISTS sentiment_analysis (
    id INTEGER PRIMARY KEY,
    text TEXT,
    sentiment TEXT
)


""


In [82]:
%sql SELECT COUNT(*) FROM sentiment_analysis;

,COUNT(*)
0,0


#Question 14


In [83]:
%%sql
INSERT INTO sentiment_analysis (text, sentiment)
VALUES
    ('This is a positive tweet', 'positive'),
    ('This is a negative tweet', 'negative'),
    ('This is a neutral tweet', 'neutral');

""


#question 15

In [84]:
%sql SELECT * FROM sentiment_analysis;

,id,text,sentiment
0,1,This is a positive tweet,positive
1,2,This is a negative tweet,negative
2,3,This is a neutral tweet,neutral


In [ ]:
# replace ###### with your file name
# make sure you have your google drive mounted.

!cp "/content/drive/MyDrive/Colab Notebooks/######.ipynb" ./
!jupyter nbconvert --to html "######.ipynb"